In [1]:
# notebooks/04_BERT_Analysis.ipynb
import os
import logging
import sys

# Configuración del entorno del notebook
notebook_dir = os.path.dirname(os.path.abspath("__file__"))
project_root = os.path.dirname(notebook_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils.file_utils import read_excel_file, save_data_to_excel
from utils.advanced_ner_utils import perform_ner_with_bert

from config import XLSX_DIRECTORY
from utils.ner_utils import cluster_entities, process_sentences_and_extract_entities
from transformers import BertTokenizer, BertForTokenClassification

# Carga de modelos y tokenizers
bert_model_name = "mrm8488/bert-spanish-cased-finetuned-ner"
bert_tokenizer = BertTokenizer.from_pretrained(bert_model_name)
bert_model = BertForTokenClassification.from_pretrained(bert_model_name)


def main():
    # Cargar los datos de las oraciones desde el archivo Excel
    logging.info("Cargando datos de oraciones desde Excel...")
    sentence_data = read_excel_file(os.path.join(XLSX_DIRECTORY, "sentence.xlsx"))

    # Realizar NER con BERT
    logging.info("Realizando NER con BERT...")
    bert_entities_df = process_sentences_and_extract_entities(
        sentence_data, bert_tokenizer, bert_model, perform_ner_with_bert, "bert"
    )

    # Convertir DataFrame de entidades a lista de diccionarios para el clustering
    bert_entities_df = bert_entities_df.to_dict("records")

    # Opcional: Agrupar entidades similares con K-means
    logging.info("Agrupando entidades similares...")
    bert_entities_df = cluster_entities(bert_entities_df, num_clusters=10)

    # Guardar los datos de entidades BERT en un nuevo archivo Excel
    logging.info("Guardando datos de entidades BERT en Excel...")
    save_data_to_excel(bert_entities_df, XLSX_DIRECTORY, "bert_entity.xlsx")


if __name__ == "__main__":
    main()

Some weights of the model checkpoint at mrm8488/bert-spanish-cased-finetuned-ner were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Procesando: 0/8534 oraciones (0.00%)

omitiendo Entidad 'Bono  Pinto' no encontrada en el texto.
Contexto: para evitar la deserción escolar [ENTIDAD] importante el Bono Juancito Pinto
Oración: Aquí fue tan importante el Bono Juancito Pinto para evitar la deserción escolar.

omitiendo Entidad 'Comandante en  de las FFAA' no encontrada en el texto.
Contexto: a todas y a todos [ENTIDAD], al Comandante en Jefe
Oración: Embajadores, organismos internacionales, al Comandante en Jefe de las FFAA, comandante de las FFAA, Comandante de la Policía Nacional; hermanos dirigentes nacionales de los distintos sectores sociales, a todas y a todos.

omitiendo Entidad 'Diagonal  Mendoza' no encontrada en el texto.
Contexto: , Tarija, hacia Potosí [ENTIDAD], hermanas, está en
Oración: Hermanos potosinos, hermanas, está en plena construcción la fábrica de cemento con un costo de más de 300 millones de dólares Podemos seguir recordando caminos; desde Oruro, Huanuni empezamos a construir el camino pavimenta

d:\TFM_Project\PLN_Project\venv\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [ ]:
# pip install transformers
# pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu111